In [2]:
import os
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt

# Number of corners in the chessboard pattern
nb_vertical = 9
nb_horizontal = 6

# Prepare object points (chessboard corners in 3D space)
objp = np.zeros((nb_horizontal * nb_vertical, 3), np.float32)
objp[:, :2] = np.mgrid[0:nb_vertical, 0:nb_horizontal].T.reshape(-1, 2)

# Arrays to store object points and image points from all the images.
objpoints = []  # 3D points in real-world space
imgpoints = []  # 2D points in the image plane.

# Collect both right and left images
right_images = glob.glob('rs/right-*.png')
left_images = glob.glob('rs/left-*.png')

# Ensure images exist for both types
assert right_images and left_images, "No images found in the specified directory."

# Create output directory for undistorted images
output_dir = 'undistorted'
os.makedirs(output_dir, exist_ok=True)

# Variable to store the first grayscale image for calibration purposes
gray_example = None

# Function to process and store calibration data
def process_images(images):
    global gray_example
    for fname in images:
        img = cv2.imread(fname)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
        # Keep the first grayscale image for calibration
        if gray_example is None:
            gray_example = gray

        # Find the chessboard corners
        ret, corners = cv2.findChessboardCorners(gray, (nb_vertical, nb_horizontal), None)
        
        # If found, add object points and image points (after refining them)
        if ret:
            objpoints.append(objp)
            imgpoints.append(corners)
            # Draw and display the corners
            img = cv2.drawChessboardCorners(img, (nb_vertical, nb_horizontal), corners, ret)
            cv2.imshow('Chessboard corners', img)
            cv2.waitKey(500)

    cv2.destroyAllWindows()

# Process the left and right images for calibration
process_images(right_images)
process_images(left_images)

# Calibration using object points and image points
# Use 'gray_example' instead of 'gray' here for calibration
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray_example.shape[::-1], None, None)

# Function to undistort and save images
def undistort_and_save(img_paths, prefix):
    saved_files = []
    for img_path in img_paths:
        img = cv2.imread(img_path)
        h, w = img.shape[:2]
        newcameramtx, roi = cv2.getOptimalNewCameraMatrix(mtx, dist, (w, h), 1, (w, h))

        # Undistort the image
        dst = cv2.undistort(img, mtx, dist, None, newcameramtx)
        
        # Save the undistorted image
        base_name = os.path.basename(img_path)
        save_path = os.path.join(output_dir, f"{prefix}_{base_name}")
        cv2.imwrite(save_path, dst)
        saved_files.append(save_path)
    
    return saved_files

# Undistort and save images
saved_right_images = undistort_and_save(right_images, "undistorted_right")
saved_left_images = undistort_and_save(left_images, "undistorted_left")

# Load and show the first undistorted image from the saved folder
def display_first_image(image_path):
    img = cv2.imread(image_path)
    plt.imshow(img[..., [2, 1, 0]])  # Convert BGR to RGB for display
    plt.title(f'Undistorted {os.path.basename(image_path)}')
    plt.show()

# Display the first saved undistorted image
display_first_image(saved_right_images[0])  # First right image
display_first_image(saved_left_images[0])   # First left image
